In [0]:
!pip install -q fake_useragent
!pip install -q gspread_pandas
!pip install -q lxml
!pip install -q gspread_dataframe

In [0]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from time import sleep
from fake_useragent import UserAgent
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings 
warnings.filterwarnings('ignore')
import datetime
import re
from gspread_pandas import Spread
from oauth2client.service_account import ServiceAccountCredentials
import gspread

In [0]:
!pip install --upgrade -q gspread

from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())


In [4]:
worksheet = gc.open('Business Development')
print(worksheet)

job_title = worksheet.worksheet("Job Title - EXCLUSIONS")
rows1 = job_title.get_all_values()

df1 = pd.DataFrame.from_records(rows1)
df1.columns = df1.iloc[0]
df1 = df1.iloc[1:]
Excluded_jobs = df1['Job Titles'].tolist()


digital = worksheet.worksheet("Digital Marketing Agencies - EXCLUSION")
row2 = digital.get_all_values()
df2 = pd.DataFrame.from_records(row2)
df2.columns = df2.iloc[0]
df2 = df2.iloc[1:]
df2['Name'] = df2['Name'].apply(lambda x: x.strip())
Digital_Mkt_names = df2['Name'].tolist()

staff = worksheet.worksheet("Staffing Agencies - EXCLUSION")
row3 = staff.get_all_values()
df3 = pd.DataFrame.from_records(row3)
df3.columns = df3.iloc[0]
df3 = df3.iloc[1:]
df3['Name'] = df3['Name'].apply(lambda x: x.strip())
Staff_Agency_names = df3['Name'].tolist()


big_company = worksheet.worksheet("Big Company - EXCLUSION")
row4 = big_company.get_all_values()
df4 = pd.DataFrame.from_records(row4)
df4.columns = df4.iloc[0]
df4 = df4.iloc[1:]
df4['Name'] = df4['Name'].apply(lambda x:x.strip())
Big_Company_names = df4['Name'].tolist()

Excluded_List = Digital_Mkt_names + Staff_Agency_names
len(Excluded_List)

<Spreadsheet 'Business Development' id:1blY5SL0qvQ9g523nEcDZ5sfhHJ2U8niVSnOb0bBSRGs>


1140

## VP Marketing

In [5]:
vpmkt_urls = []
vpmkt_info = []
vpmkt_job_title = []
user_agent = UserAgent()

for i in range(30):
    print("Scraping Page:",i+1)
    main_url = 'https://www.glassdoor.com/Job/us-vp-marketing-jobs-SRCH_IL.0,2_IN1_KO3,15_IP{}.htm?fromAge=1'.format(i+1)

    page = requests.get(main_url,headers={'user-agent':user_agent.chrome})

    soup = BeautifulSoup(page.content,'lxml')

    all_divs = soup.select('li.jl div div div a.jobLink')
    if all_divs != []:
        for div in all_divs:
            vpmkt_urls.append(div['href'])
            vpmkt_job_title.append(div.get_text())

    sleep(np.random.randint(5))

    all_divs1 = soup.select('li.jl div div.flexbox.empLoc div')
    if all_divs1 != []:
        for div in all_divs1:
            if div.get_text() != 'New' and div.get_text() != "We're Hiring" and div.get_text() != "EASY APPLY" and div.get_text() != "Hot" and div.get_text() != 'Award Winner':
              vpmkt_info.append(div.get_text())

                     
        
    sleep(np.random.randint(5))

Scraping Page: 1
Scraping Page: 2
Scraping Page: 3
Scraping Page: 4
Scraping Page: 5
Scraping Page: 6
Scraping Page: 7
Scraping Page: 8
Scraping Page: 9
Scraping Page: 10
Scraping Page: 11
Scraping Page: 12
Scraping Page: 13
Scraping Page: 14
Scraping Page: 15
Scraping Page: 16
Scraping Page: 17
Scraping Page: 18
Scraping Page: 19
Scraping Page: 20
Scraping Page: 21
Scraping Page: 22
Scraping Page: 23
Scraping Page: 24
Scraping Page: 25
Scraping Page: 26
Scraping Page: 27
Scraping Page: 28
Scraping Page: 29
Scraping Page: 30


In [6]:
len(vpmkt_urls)
len(vpmkt_job_title)
len(vpmkt_info)

565

565

565

In [0]:
vp_mkt_urls = []
base_link = 'http://www.glassdoor.com'
for link in vpmkt_urls:
    vp_mkt_urls.append(base_link + link)

In [8]:
vpmkt_df = pd.DataFrame({'Job_Title':vpmkt_job_title,"INFO":vpmkt_info,"URLs":vp_mkt_urls})
vpmkt_df = vpmkt_df[~(vpmkt_df['Job_Title'].str.contains("None"))]
vpmkt_df = vpmkt_df[~(vpmkt_df['INFO'].str.contains("None"))]
vpmkt_df['Company_Name'] = vpmkt_df['INFO'].apply(lambda x: x.strip().split("–")[0].strip())
vpmkt_df['Location'] = vpmkt_df['INFO'].apply(lambda x: x.strip().split("–")[1].strip())
vpmkt_df = vpmkt_df.drop("INFO",axis=1)
vpmkt_df = vpmkt_df.reset_index(drop=True)
vpmkt_df.head()

,Job_Title,URLs,Company_Name,Location
0,Director of Marketing,http://www.glassdoor.com/partner/jobListing.ht...,LandaJob Advertising & Marketing Talent,"Kansas City, MO"
1,VP Marketing,http://www.glassdoor.com/partner/jobListing.ht...,Lai Ventures - For Early Stage Client,"Scottsdale, AZ"
2,VP of eCommerce Marketing,http://www.glassdoor.com/partner/jobListing.ht...,RealTruck,"Ocala, FL"
3,Marketing Analyst,http://www.glassdoor.com/partner/jobListing.ht...,Owlet Baby Care,"Lehi, UT"
4,"VP, Marketing",http://www.glassdoor.com/partner/jobListing.ht...,Walt Disney Company,"Burbank, CA"


In [9]:
vpmkt_df = vpmkt_df.drop_duplicates(subset=['Company_Name']).reset_index(drop=True)
len(vpmkt_df)
vpmkt_df.head()
vpmkt_df['Job_Title'] = vpmkt_df['Job_Title'].apply(lambda x: x if "Marketing" in x else 'None')
vpmkt_df = vpmkt_df[vpmkt_df['Job_Title'] != 'None'].reset_index(drop=True)
len(vpmkt_df)
vpmkt_df.head()

456

,Job_Title,URLs,Company_Name,Location
0,Director of Marketing,http://www.glassdoor.com/partner/jobListing.ht...,LandaJob Advertising & Marketing Talent,"Kansas City, MO"
1,VP Marketing,http://www.glassdoor.com/partner/jobListing.ht...,Lai Ventures - For Early Stage Client,"Scottsdale, AZ"
2,VP of eCommerce Marketing,http://www.glassdoor.com/partner/jobListing.ht...,RealTruck,"Ocala, FL"
3,Marketing Analyst,http://www.glassdoor.com/partner/jobListing.ht...,Owlet Baby Care,"Lehi, UT"
4,"VP, Marketing",http://www.glassdoor.com/partner/jobListing.ht...,Walt Disney Company,"Burbank, CA"


280

,Job_Title,URLs,Company_Name,Location
0,Director of Marketing,http://www.glassdoor.com/partner/jobListing.ht...,LandaJob Advertising & Marketing Talent,"Kansas City, MO"
1,VP Marketing,http://www.glassdoor.com/partner/jobListing.ht...,Lai Ventures - For Early Stage Client,"Scottsdale, AZ"
2,VP of eCommerce Marketing,http://www.glassdoor.com/partner/jobListing.ht...,RealTruck,"Ocala, FL"
3,Marketing Analyst,http://www.glassdoor.com/partner/jobListing.ht...,Owlet Baby Care,"Lehi, UT"
4,"VP, Marketing",http://www.glassdoor.com/partner/jobListing.ht...,Walt Disney Company,"Burbank, CA"


In [10]:
vpmkt_df['Contact Email'] = np.nan
vpmkt_df['Contact Name'] = np.nan
vpmkt_df['Contact Position'] = np.nan
vpmkt_df['Linkedin Sent Date'] = np.nan
vpmkt_df['Date 1st Email Sent'] = np.nan
vpmkt_df['date'] = str(datetime.datetime.now())
vpmkt_df.head()
len(vpmkt_df)

,Job_Title,URLs,Company_Name,Location,Contact Email,Contact Name,Contact Position,Linkedin Sent Date,Date 1st Email Sent,date
0,Director of Marketing,http://www.glassdoor.com/partner/jobListing.ht...,LandaJob Advertising & Marketing Talent,"Kansas City, MO",NaN,NaN,NaN,NaN,NaN,2018-08-15 17:28:16.510522
1,VP Marketing,http://www.glassdoor.com/partner/jobListing.ht...,Lai Ventures - For Early Stage Client,"Scottsdale, AZ",NaN,NaN,NaN,NaN,NaN,2018-08-15 17:28:16.510522
2,VP of eCommerce Marketing,http://www.glassdoor.com/partner/jobListing.ht...,RealTruck,"Ocala, FL",NaN,NaN,NaN,NaN,NaN,2018-08-15 17:28:16.510522
3,Marketing Analyst,http://www.glassdoor.com/partner/jobListing.ht...,Owlet Baby Care,"Lehi, UT",NaN,NaN,NaN,NaN,NaN,2018-08-15 17:28:16.510522
4,"VP, Marketing",http://www.glassdoor.com/partner/jobListing.ht...,Walt Disney Company,"Burbank, CA",NaN,NaN,NaN,NaN,NaN,2018-08-15 17:28:16.510522


280

## Digital Marketing

In [11]:
digitalmkt_urls = []
digitalmkt_info = []
digitalmkt_job_title = []
user_agent = UserAgent()

for i in range(30):
    print("Scraping Page:",i+1)
    main_url = 'https://www.glassdoor.com/Job/us-digital-marketing-jobs-SRCH_IL.0,2_IN1_KO3,20_IP{}.htm?fromAge=1'.format(i+1)

    page = requests.get(main_url,headers={'user-agent':user_agent.chrome})

    soup = BeautifulSoup(page.content,'lxml')

    all_divs = soup.select('li.jl div div div a.jobLink')
    if all_divs != []:
        for div in all_divs:
            digitalmkt_urls.append(div['href'])
            digitalmkt_job_title.append(div.get_text())
            
    sleep(np.random.randint(5))

    all_divs1 = soup.select('li.jl div div.flexbox.empLoc div')
    if all_divs1 != []:
        for div in all_divs1:
            if div.get_text() != 'New' and div.get_text() != "We're Hiring" and div.get_text() != "EASY APPLY" and div.get_text() != "Hot" and div.get_text() != 'Award Winner':
               digitalmkt_info.append(div.get_text())

                      
    sleep(np.random.randint(5))

Scraping Page: 1
Scraping Page: 2
Scraping Page: 3
Scraping Page: 4
Scraping Page: 5
Scraping Page: 6
Scraping Page: 7
Scraping Page: 8
Scraping Page: 9
Scraping Page: 10
Scraping Page: 11
Scraping Page: 12
Scraping Page: 13
Scraping Page: 14
Scraping Page: 15
Scraping Page: 16
Scraping Page: 17
Scraping Page: 18
Scraping Page: 19
Scraping Page: 20
Scraping Page: 21
Scraping Page: 22
Scraping Page: 23
Scraping Page: 24
Scraping Page: 25
Scraping Page: 26
Scraping Page: 27
Scraping Page: 28
Scraping Page: 29
Scraping Page: 30


In [12]:
len(digitalmkt_urls)
len(digitalmkt_job_title)
len(digitalmkt_info)

870

870

870

In [0]:
digital_mkt_urls = []
base_link = 'http://www.glassdoor.com'
for link in digitalmkt_urls:
    digital_mkt_urls.append(base_link + link)

In [14]:
digitalmkt_df = pd.DataFrame({'Job_Title':digitalmkt_job_title,"INFO":digitalmkt_info,"URLs":digital_mkt_urls})
digitalmkt_df = digitalmkt_df[~(digitalmkt_df['Job_Title'].str.contains("None"))]
digitalmkt_df = digitalmkt_df[~(digitalmkt_df['INFO'].str.contains("None"))]
digitalmkt_df['Company_Name'] = digitalmkt_df['INFO'].apply(lambda x: x.strip().split("–")[0].strip())
digitalmkt_df['Location'] = digitalmkt_df['INFO'].apply(lambda x: x.strip().split("–")[1].strip())
digitalmkt_df = digitalmkt_df.drop("INFO",axis=1)
digitalmkt_df = digitalmkt_df.reset_index(drop=True)
digitalmkt_df.head()

,Job_Title,URLs,Company_Name,Location
0,Digital Marketing Recruiter,http://www.glassdoor.com/partner/jobListing.ht...,Jellyfish,"Baltimore, MD"
1,Digital Marketing Manager,http://www.glassdoor.com/partner/jobListing.ht...,Clayco,"Saint Louis, MO"
2,Director of Marketing,http://www.glassdoor.com/partner/jobListing.ht...,LandaJob Advertising & Marketing Talent,"Kansas City, MO"
3,SEO Specialist,http://www.glassdoor.com/partner/jobListing.ht...,Veterans United Home Loans,"Columbia, MO"
4,Digital Marketing & Public Relations Specialist,http://www.glassdoor.com/partner/jobListing.ht...,Inseev Interactive,"San Diego, CA"


In [15]:
digitalmkt_df = digitalmkt_df.drop_duplicates(subset=['Company_Name']).reset_index(drop=True)
len(digitalmkt_df)
digitalmkt_df.head()
digitalmkt_df['Job_Title'] = digitalmkt_df['Job_Title'].apply(lambda x: x if "Marketing" in x else 'None')
digitalmkt_df = digitalmkt_df[digitalmkt_df['Job_Title'] != 'None'].reset_index(drop=True)
len(digitalmkt_df)
digitalmkt_df.head()

678

,Job_Title,URLs,Company_Name,Location
0,Digital Marketing Recruiter,http://www.glassdoor.com/partner/jobListing.ht...,Jellyfish,"Baltimore, MD"
1,Digital Marketing Manager,http://www.glassdoor.com/partner/jobListing.ht...,Clayco,"Saint Louis, MO"
2,Director of Marketing,http://www.glassdoor.com/partner/jobListing.ht...,LandaJob Advertising & Marketing Talent,"Kansas City, MO"
3,SEO Specialist,http://www.glassdoor.com/partner/jobListing.ht...,Veterans United Home Loans,"Columbia, MO"
4,Digital Marketing & Public Relations Specialist,http://www.glassdoor.com/partner/jobListing.ht...,Inseev Interactive,"San Diego, CA"


419

,Job_Title,URLs,Company_Name,Location
0,Digital Marketing Recruiter,http://www.glassdoor.com/partner/jobListing.ht...,Jellyfish,"Baltimore, MD"
1,Digital Marketing Manager,http://www.glassdoor.com/partner/jobListing.ht...,Clayco,"Saint Louis, MO"
2,Director of Marketing,http://www.glassdoor.com/partner/jobListing.ht...,LandaJob Advertising & Marketing Talent,"Kansas City, MO"
3,Digital Marketing & Public Relations Specialist,http://www.glassdoor.com/partner/jobListing.ht...,Inseev Interactive,"San Diego, CA"
4,Digital Marketing Manager,http://www.glassdoor.com/partner/jobListing.ht...,Astronova,"West Warwick, RI"


In [16]:
digitalmkt_df['Contact Email'] = np.nan
digitalmkt_df['Contact Name'] = np.nan
digitalmkt_df['Contact Position'] = np.nan
digitalmkt_df['Linkedin Sent Date'] = np.nan
digitalmkt_df['Date 1st Email Sent'] = np.nan
digitalmkt_df['date'] = str(datetime.datetime.now())
digitalmkt_df.head()
len(digitalmkt_df)

,Job_Title,URLs,Company_Name,Location,Contact Email,Contact Name,Contact Position,Linkedin Sent Date,Date 1st Email Sent,date
0,Digital Marketing Recruiter,http://www.glassdoor.com/partner/jobListing.ht...,Jellyfish,"Baltimore, MD",NaN,NaN,NaN,NaN,NaN,2018-08-15 17:32:18.502058
1,Digital Marketing Manager,http://www.glassdoor.com/partner/jobListing.ht...,Clayco,"Saint Louis, MO",NaN,NaN,NaN,NaN,NaN,2018-08-15 17:32:18.502058
2,Director of Marketing,http://www.glassdoor.com/partner/jobListing.ht...,LandaJob Advertising & Marketing Talent,"Kansas City, MO",NaN,NaN,NaN,NaN,NaN,2018-08-15 17:32:18.502058
3,Digital Marketing & Public Relations Specialist,http://www.glassdoor.com/partner/jobListing.ht...,Inseev Interactive,"San Diego, CA",NaN,NaN,NaN,NaN,NaN,2018-08-15 17:32:18.502058
4,Digital Marketing Manager,http://www.glassdoor.com/partner/jobListing.ht...,Astronova,"West Warwick, RI",NaN,NaN,NaN,NaN,NaN,2018-08-15 17:32:18.502058


419

## Ecommerce

In [17]:
ecommerce_urls = []
ecommerce_info = []
ecommerce_job_title = []
user_agent = UserAgent()

for i in range(30):
    print("Scraping Page:",i+1)
    main_url = 'https://www.glassdoor.com/Job/us-ecommerce-jobs-SRCH_IL.0,2_IN1_KO3,12_IP{}.htm?fromAge=1'.format(i+1)

    page = requests.get(main_url,headers={'user-agent':user_agent.chrome})

    soup = BeautifulSoup(page.content,'lxml')

    all_divs = soup.select('li.jl div div div a.jobLink')
    if all_divs != []:
        for div in all_divs:
            ecommerce_urls.append(div['href'])
            ecommerce_job_title.append(div.get_text())

    sleep(np.random.randint(5))
    
    all_divs1 = soup.select('li.jl div div.flexbox.empLoc div')
    if all_divs1 != []:
        for div in all_divs1:
            if div.get_text() != 'New' and div.get_text() != "We're Hiring" and div.get_text() != "EASY APPLY" and div.get_text() != "Hot" and div.get_text() != 'Award Winner':
                ecommerce_info.append(div.get_text())

                      
    sleep(np.random.randint(5))

Scraping Page: 1
Scraping Page: 2
Scraping Page: 3
Scraping Page: 4
Scraping Page: 5
Scraping Page: 6
Scraping Page: 7
Scraping Page: 8
Scraping Page: 9
Scraping Page: 10
Scraping Page: 11
Scraping Page: 12
Scraping Page: 13
Scraping Page: 14
Scraping Page: 15
Scraping Page: 16
Scraping Page: 17
Scraping Page: 18
Scraping Page: 19
Scraping Page: 20
Scraping Page: 21
Scraping Page: 22
Scraping Page: 23
Scraping Page: 24
Scraping Page: 25
Scraping Page: 26
Scraping Page: 27
Scraping Page: 28
Scraping Page: 29
Scraping Page: 30


In [18]:
len(ecommerce_urls)
len(ecommerce_job_title)
len(ecommerce_info)

899

899

899

In [0]:
ecommerce = []
base_link = 'http://www.glassdoor.com'
for link in ecommerce_urls:
    ecommerce.append(base_link + link)

In [20]:
ecommerce_df = pd.DataFrame({'Job_Title':ecommerce_job_title,"INFO":ecommerce_info,"URLs":ecommerce})
ecommerce_df = ecommerce_df[~(ecommerce_df['Job_Title'].str.contains("None"))]
ecommerce_df = ecommerce_df[~(ecommerce_df['INFO'].str.contains("None"))]
ecommerce_df['Company_Name'] = ecommerce_df['INFO'].apply(lambda x: x.strip().split("–")[0].strip())
ecommerce_df['Location'] = ecommerce_df['INFO'].apply(lambda x: x.strip().split("–")[1].strip())
ecommerce_df = ecommerce_df.drop("INFO",axis=1)
ecommerce_df = ecommerce_df.reset_index(drop=True)
ecommerce_df.head()

,Job_Title,URLs,Company_Name,Location
0,eCommerce Manager,http://www.glassdoor.com/partner/jobListing.ht...,Fontem Ventures,"Charlotte, NC"
1,Program Manager,http://www.glassdoor.com/partner/jobListing.ht...,Snapfish,"Fort Collins, CO"
2,Sales Associate eCommerce and Phone- AutoNatio...,http://www.glassdoor.com/partner/jobListing.ht...,AutoNation,"Panama City, FL"
3,"Customer Engagement Manager, eCommerce",http://www.glassdoor.com/partner/jobListing.ht...,LiveArea,"Allen, TX"
4,Customer Experience Associate,http://www.glassdoor.com/partner/jobListing.ht...,"Rothy's, Inc.","San Francisco, CA"


In [21]:
ecommerce_df = ecommerce_df.drop_duplicates(subset=['Company_Name']).reset_index(drop=True)
len(ecommerce_df)
ecommerce_df.head()
ecommerce_df['Job_Title'] = ecommerce_df['Job_Title'].apply(lambda x: x if "Marketing" in x else 'None')
ecommerce_df = ecommerce_df[ecommerce_df['Job_Title'] != 'None'].reset_index(drop=True)
len(ecommerce_df)
ecommerce_df.head()

588

,Job_Title,URLs,Company_Name,Location
0,eCommerce Manager,http://www.glassdoor.com/partner/jobListing.ht...,Fontem Ventures,"Charlotte, NC"
1,Program Manager,http://www.glassdoor.com/partner/jobListing.ht...,Snapfish,"Fort Collins, CO"
2,Sales Associate eCommerce and Phone- AutoNatio...,http://www.glassdoor.com/partner/jobListing.ht...,AutoNation,"Panama City, FL"
3,"Customer Engagement Manager, eCommerce",http://www.glassdoor.com/partner/jobListing.ht...,LiveArea,"Allen, TX"
4,Customer Experience Associate,http://www.glassdoor.com/partner/jobListing.ht...,"Rothy's, Inc.","San Francisco, CA"


64

,Job_Title,URLs,Company_Name,Location
0,VP of eCommerce Marketing,http://www.glassdoor.com/partner/jobListing.ht...,RealTruck,"Ocala, FL"
1,Audience Development Marketing Manager,http://www.glassdoor.com/partner/jobListing.ht...,CAS,"Washington, DC"
2,Internet Marketing Specialist,http://www.glassdoor.com/partner/jobListing.ht...,eBay,"San Jose, CA"
3,Digital Marketing Specialist,http://www.glassdoor.com/partner/jobListing.ht...,Trupanion,"Seattle, WA"
4,eCommerce Marketing Manager,http://www.glassdoor.com/partner/jobListing.ht...,Salix Pharmaceuticals,"Bridgewater, NJ"


In [22]:
ecommerce_df['Contact Email'] = np.nan
ecommerce_df['Contact Name'] = np.nan
ecommerce_df['Contact Position'] = np.nan
ecommerce_df['Linkedin Sent Date'] = np.nan
ecommerce_df['Date 1st Email Sent'] = np.nan
ecommerce_df['date'] = str(datetime.datetime.now())
ecommerce_df.head()
len(ecommerce_df)

,Job_Title,URLs,Company_Name,Location,Contact Email,Contact Name,Contact Position,Linkedin Sent Date,Date 1st Email Sent,date
0,VP of eCommerce Marketing,http://www.glassdoor.com/partner/jobListing.ht...,RealTruck,"Ocala, FL",NaN,NaN,NaN,NaN,NaN,2018-08-15 17:35:48.491174
1,Audience Development Marketing Manager,http://www.glassdoor.com/partner/jobListing.ht...,CAS,"Washington, DC",NaN,NaN,NaN,NaN,NaN,2018-08-15 17:35:48.491174
2,Internet Marketing Specialist,http://www.glassdoor.com/partner/jobListing.ht...,eBay,"San Jose, CA",NaN,NaN,NaN,NaN,NaN,2018-08-15 17:35:48.491174
3,Digital Marketing Specialist,http://www.glassdoor.com/partner/jobListing.ht...,Trupanion,"Seattle, WA",NaN,NaN,NaN,NaN,NaN,2018-08-15 17:35:48.491174
4,eCommerce Marketing Manager,http://www.glassdoor.com/partner/jobListing.ht...,Salix Pharmaceuticals,"Bridgewater, NJ",NaN,NaN,NaN,NaN,NaN,2018-08-15 17:35:48.491174


64

## Digital Marketing Associate

In [23]:
digitalmkt_asso_urls = []
digitalmkt_asso_info = []
digitalmkt_asso_job_title = []
user_agent = UserAgent()

for i in range(30):
    print("Scraping Page:",i+1)
    main_url = 'https://www.glassdoor.com/Job/us-digital-marketing-associate-jobs-SRCH_IL.0,2_IN1_KO3,30_IP{}.htm?fromAge=1'.format(i+1)

    page = requests.get(main_url,headers={'user-agent':user_agent.chrome})

    soup = BeautifulSoup(page.content,'lxml')

    all_divs = soup.select('li.jl div div div a.jobLink')
    if all_divs != []:
        for div in all_divs:
            digitalmkt_asso_urls.append(div['href'])
            digitalmkt_asso_job_title.append(div.get_text())

    sleep(np.random.randint(5))
    

    all_divs1 = soup.select('li.jl div div.flexbox.empLoc div')
    if all_divs1 != []:
        for div in all_divs1:
            if div.get_text() != 'New' and div.get_text() != "We're Hiring" and div.get_text() != "EASY APPLY" and div.get_text() != "Hot" and div.get_text() != 'Award Winner':
                digitalmkt_asso_info.append(div.get_text())

                      
    sleep(np.random.randint(5))

Scraping Page: 1
Scraping Page: 2
Scraping Page: 3
Scraping Page: 4
Scraping Page: 5
Scraping Page: 6
Scraping Page: 7
Scraping Page: 8
Scraping Page: 9
Scraping Page: 10
Scraping Page: 11
Scraping Page: 12
Scraping Page: 13
Scraping Page: 14
Scraping Page: 15
Scraping Page: 16
Scraping Page: 17
Scraping Page: 18
Scraping Page: 19
Scraping Page: 20
Scraping Page: 21
Scraping Page: 22
Scraping Page: 23
Scraping Page: 24
Scraping Page: 25
Scraping Page: 26
Scraping Page: 27
Scraping Page: 28
Scraping Page: 29
Scraping Page: 30


In [24]:
len(digitalmkt_asso_urls)
len(digitalmkt_asso_job_title)
len(digitalmkt_asso_info)

772

772

772

In [0]:
digitalmkt_asso = []
base_link = 'http://www.glassdoor.com'
for link in digitalmkt_asso_urls:
    digitalmkt_asso.append(base_link + link)

In [26]:
digitalmkt_asso_df = pd.DataFrame({'Job_Title':digitalmkt_asso_job_title,"INFO":digitalmkt_asso_info,"URLs":digitalmkt_asso})
digitalmkt_asso_df = digitalmkt_asso_df[~(digitalmkt_asso_df['Job_Title'].str.contains("None"))]
digitalmkt_asso_df = digitalmkt_asso_df[~(digitalmkt_asso_df['INFO'].str.contains("None"))]
digitalmkt_asso_df['Company_Name'] = digitalmkt_asso_df['INFO'].apply(lambda x: x.strip().split("–")[0].strip())
digitalmkt_asso_df['Location'] = digitalmkt_asso_df['INFO'].apply(lambda x: x.strip().split("–")[1].strip())
digitalmkt_asso_df = digitalmkt_asso_df.drop("INFO",axis=1)
digitalmkt_asso_df = digitalmkt_asso_df.reset_index(drop=True)
digitalmkt_asso_df.head()

,Job_Title,URLs,Company_Name,Location
0,Digital Marketing Recruiter,http://www.glassdoor.com/partner/jobListing.ht...,Jellyfish,"Baltimore, MD"
1,Marketing Associate,http://www.glassdoor.com/partner/jobListing.ht...,Entefy,"Palo Alto, CA"
2,Digital Marketing Manager,http://www.glassdoor.com/partner/jobListing.ht...,Clayco,"Saint Louis, MO"
3,Digital Marketing Specialist,http://www.glassdoor.com/partner/jobListing.ht...,Trupanion,"Seattle, WA"
4,Manager of Digital Data & Analytics,http://www.glassdoor.com/partner/jobListing.ht...,Chattem,"Chattanooga, TN"


In [27]:
digitalmkt_asso_df = digitalmkt_asso_df.drop_duplicates(subset=['Company_Name']).reset_index(drop=True)
len(digitalmkt_asso_df)
digitalmkt_asso_df.head()
digitalmkt_asso_df['Job_Title'] = digitalmkt_asso_df['Job_Title'].apply(lambda x: x if "Marketing" in x else 'None')
digitalmkt_asso_df = digitalmkt_asso_df[digitalmkt_asso_df['Job_Title'] != 'None'].reset_index(drop=True)
len(digitalmkt_asso_df)
digitalmkt_asso_df.head()

573

,Job_Title,URLs,Company_Name,Location
0,Digital Marketing Recruiter,http://www.glassdoor.com/partner/jobListing.ht...,Jellyfish,"Baltimore, MD"
1,Marketing Associate,http://www.glassdoor.com/partner/jobListing.ht...,Entefy,"Palo Alto, CA"
2,Digital Marketing Manager,http://www.glassdoor.com/partner/jobListing.ht...,Clayco,"Saint Louis, MO"
3,Digital Marketing Specialist,http://www.glassdoor.com/partner/jobListing.ht...,Trupanion,"Seattle, WA"
4,Manager of Digital Data & Analytics,http://www.glassdoor.com/partner/jobListing.ht...,Chattem,"Chattanooga, TN"


362

,Job_Title,URLs,Company_Name,Location
0,Digital Marketing Recruiter,http://www.glassdoor.com/partner/jobListing.ht...,Jellyfish,"Baltimore, MD"
1,Marketing Associate,http://www.glassdoor.com/partner/jobListing.ht...,Entefy,"Palo Alto, CA"
2,Digital Marketing Manager,http://www.glassdoor.com/partner/jobListing.ht...,Clayco,"Saint Louis, MO"
3,Digital Marketing Specialist,http://www.glassdoor.com/partner/jobListing.ht...,Trupanion,"Seattle, WA"
4,"Digital Marketing, Senior Associate",http://www.glassdoor.com/partner/jobListing.ht...,Remitly,"Seattle, WA"


In [28]:
digitalmkt_asso_df['Contact Email'] = np.nan
digitalmkt_asso_df['Contact Name'] = np.nan
digitalmkt_asso_df['Contact Position'] = np.nan
digitalmkt_asso_df['Linkedin Sent Date'] = np.nan
digitalmkt_asso_df['Date 1st Email Sent'] = np.nan
digitalmkt_asso_df['date'] = str(datetime.datetime.now())
digitalmkt_asso_df.head()
len(digitalmkt_asso_df)

,Job_Title,URLs,Company_Name,Location,Contact Email,Contact Name,Contact Position,Linkedin Sent Date,Date 1st Email Sent,date
0,Digital Marketing Recruiter,http://www.glassdoor.com/partner/jobListing.ht...,Jellyfish,"Baltimore, MD",NaN,NaN,NaN,NaN,NaN,2018-08-15 17:39:10.494665
1,Marketing Associate,http://www.glassdoor.com/partner/jobListing.ht...,Entefy,"Palo Alto, CA",NaN,NaN,NaN,NaN,NaN,2018-08-15 17:39:10.494665
2,Digital Marketing Manager,http://www.glassdoor.com/partner/jobListing.ht...,Clayco,"Saint Louis, MO",NaN,NaN,NaN,NaN,NaN,2018-08-15 17:39:10.494665
3,Digital Marketing Specialist,http://www.glassdoor.com/partner/jobListing.ht...,Trupanion,"Seattle, WA",NaN,NaN,NaN,NaN,NaN,2018-08-15 17:39:10.494665
4,"Digital Marketing, Senior Associate",http://www.glassdoor.com/partner/jobListing.ht...,Remitly,"Seattle, WA",NaN,NaN,NaN,NaN,NaN,2018-08-15 17:39:10.494665


362

## PPC

In [29]:
ppc_urls = []
ppc_info = []
ppc_job_title = []
user_agent = UserAgent()

for i in range(30):
    print("Scraping Page:",i+1)
    main_url = 'https://www.glassdoor.com/Job/us-digital-marketing-associate-jobs-SRCH_IL.0,2_IN1_KO3,30_IP{}.htm?fromAge=1'.format(i+1)

    page = requests.get(main_url,headers={'user-agent':user_agent.chrome})

    soup = BeautifulSoup(page.content,'lxml')

    all_divs = soup.select('li.jl div div div a.jobLink')
    if all_divs != []:
        for div in all_divs:
            ppc_urls.append(div['href'])
            ppc_job_title.append(div.get_text())

    sleep(np.random.randint(5))
    

    all_divs1 = soup.select('li.jl div div.flexbox.empLoc div')
    if all_divs1 != []:
        for div in all_divs1:
            if div.get_text() != 'New' and div.get_text() != "We're Hiring" and div.get_text() != "EASY APPLY" and div.get_text() != "Hot" and div.get_text() != 'Award Winner':
                ppc_info.append(div.get_text())

                      
    sleep(np.random.randint(5))

Scraping Page: 1
Scraping Page: 2
Scraping Page: 3
Scraping Page: 4
Scraping Page: 5
Scraping Page: 6
Scraping Page: 7
Scraping Page: 8
Scraping Page: 9
Scraping Page: 10
Scraping Page: 11
Scraping Page: 12
Scraping Page: 13
Scraping Page: 14
Scraping Page: 15
Scraping Page: 16
Scraping Page: 17
Scraping Page: 18
Scraping Page: 19
Scraping Page: 20
Scraping Page: 21
Scraping Page: 22
Scraping Page: 23
Scraping Page: 24
Scraping Page: 25
Scraping Page: 26
Scraping Page: 27
Scraping Page: 28
Scraping Page: 29
Scraping Page: 30


In [30]:
len(ppc_urls)
len(ppc_job_title)
len(ppc_info)

778

778

778

In [0]:
ppc = []
base_link = 'http://www.glassdoor.com'
for link in ppc_urls:
    ppc.append(base_link + link)

In [32]:
ppc_df = pd.DataFrame({'Job_Title':ppc_job_title,"INFO":ppc_info,"URLs":ppc})
ppc_df = ppc_df[~(ppc_df['Job_Title'].str.contains("None"))]
ppc_df = ppc_df[~(ppc_df['INFO'].str.contains("None"))]
ppc_df['Company_Name'] = ppc_df['INFO'].apply(lambda x: x.strip().split("–")[0].strip())
ppc_df['Location'] = ppc_df['INFO'].apply(lambda x: x.strip().split("–")[1].strip())
ppc_df = ppc_df.drop("INFO",axis=1)
ppc_df = ppc_df.reset_index(drop=True)
ppc_df.head()

,Job_Title,URLs,Company_Name,Location
0,Digital Marketing Recruiter,http://www.glassdoor.com/partner/jobListing.ht...,Jellyfish,"Baltimore, MD"
1,Digital Marketing Manager,http://www.glassdoor.com/partner/jobListing.ht...,Clayco,"Saint Louis, MO"
2,Marketing Associate,http://www.glassdoor.com/partner/jobListing.ht...,Entefy,"Palo Alto, CA"
3,Digital Marketing Specialist,http://www.glassdoor.com/partner/jobListing.ht...,Trupanion,"Seattle, WA"
4,Manager of Digital Data & Analytics,http://www.glassdoor.com/partner/jobListing.ht...,Chattem,"Chattanooga, TN"


In [33]:
ppc_df = ppc_df.drop_duplicates(subset=['Company_Name']).reset_index(drop=True)
len(ppc_df)
ppc_df.head()
ppc_df['Job_Title'] = ppc_df['Job_Title'].apply(lambda x: x if "Marketing" in x else 'None')
ppc_df = ppc_df[ppc_df['Job_Title'] != 'None'].reset_index(drop=True)
len(ppc_df)
ppc_df.head()

593

,Job_Title,URLs,Company_Name,Location
0,Digital Marketing Recruiter,http://www.glassdoor.com/partner/jobListing.ht...,Jellyfish,"Baltimore, MD"
1,Digital Marketing Manager,http://www.glassdoor.com/partner/jobListing.ht...,Clayco,"Saint Louis, MO"
2,Marketing Associate,http://www.glassdoor.com/partner/jobListing.ht...,Entefy,"Palo Alto, CA"
3,Digital Marketing Specialist,http://www.glassdoor.com/partner/jobListing.ht...,Trupanion,"Seattle, WA"
4,Manager of Digital Data & Analytics,http://www.glassdoor.com/partner/jobListing.ht...,Chattem,"Chattanooga, TN"


375

,Job_Title,URLs,Company_Name,Location
0,Digital Marketing Recruiter,http://www.glassdoor.com/partner/jobListing.ht...,Jellyfish,"Baltimore, MD"
1,Digital Marketing Manager,http://www.glassdoor.com/partner/jobListing.ht...,Clayco,"Saint Louis, MO"
2,Marketing Associate,http://www.glassdoor.com/partner/jobListing.ht...,Entefy,"Palo Alto, CA"
3,Digital Marketing Specialist,http://www.glassdoor.com/partner/jobListing.ht...,Trupanion,"Seattle, WA"
4,"Digital Marketing, Senior Associate",http://www.glassdoor.com/partner/jobListing.ht...,Remitly,"Seattle, WA"


In [34]:
ppc_df['Contact Email'] = np.nan
ppc_df['Contact Name'] = np.nan
ppc_df['Contact Position'] = np.nan
ppc_df['Linkedin Sent Date'] = np.nan
ppc_df['Date 1st Email Sent'] = np.nan
ppc_df['date'] = str(datetime.datetime.now())
ppc_df.head()
len(ppc_df)

,Job_Title,URLs,Company_Name,Location,Contact Email,Contact Name,Contact Position,Linkedin Sent Date,Date 1st Email Sent,date
0,Digital Marketing Recruiter,http://www.glassdoor.com/partner/jobListing.ht...,Jellyfish,"Baltimore, MD",NaN,NaN,NaN,NaN,NaN,2018-08-15 17:42:31.679386
1,Digital Marketing Manager,http://www.glassdoor.com/partner/jobListing.ht...,Clayco,"Saint Louis, MO",NaN,NaN,NaN,NaN,NaN,2018-08-15 17:42:31.679386
2,Marketing Associate,http://www.glassdoor.com/partner/jobListing.ht...,Entefy,"Palo Alto, CA",NaN,NaN,NaN,NaN,NaN,2018-08-15 17:42:31.679386
3,Digital Marketing Specialist,http://www.glassdoor.com/partner/jobListing.ht...,Trupanion,"Seattle, WA",NaN,NaN,NaN,NaN,NaN,2018-08-15 17:42:31.679386
4,"Digital Marketing, Senior Associate",http://www.glassdoor.com/partner/jobListing.ht...,Remitly,"Seattle, WA",NaN,NaN,NaN,NaN,NaN,2018-08-15 17:42:31.679386


375

## SEM

In [35]:
sem_urls = []
sem_info = []
sem_job_title = []
user_agent = UserAgent()

for i in range(30):
    print("Scraping Page:",i+1)
    main_url = 'https://www.glassdoor.com/Job/us-digital-marketing-associate-jobs-SRCH_IL.0,2_IN1_KO3,30_IP{}.htm?fromAge=1'.format(i+1)

    page = requests.get(main_url,headers={'user-agent':user_agent.chrome})

    soup = BeautifulSoup(page.content,'lxml')

    all_divs = soup.select('li.jl div div div a.jobLink')
    if all_divs != []:
        for div in all_divs:
            sem_urls.append(div['href'])
            sem_job_title.append(div.get_text())

    sleep(np.random.randint(5))
    

    all_divs1 = soup.select('li.jl div div.flexbox.empLoc div')
    if all_divs1 != []:
        for div in all_divs1:
            if div.get_text() != 'New' and div.get_text() != "We're Hiring" and div.get_text() != "EASY APPLY" and div.get_text() != "Hot" and div.get_text() != 'Award Winner':
              sem_info.append(div.get_text())

                      
    sleep(np.random.randint(5))

Scraping Page: 1
Scraping Page: 2
Scraping Page: 3
Scraping Page: 4
Scraping Page: 5
Scraping Page: 6
Scraping Page: 7
Scraping Page: 8
Scraping Page: 9
Scraping Page: 10
Scraping Page: 11
Scraping Page: 12
Scraping Page: 13
Scraping Page: 14
Scraping Page: 15
Scraping Page: 16
Scraping Page: 17
Scraping Page: 18
Scraping Page: 19
Scraping Page: 20
Scraping Page: 21
Scraping Page: 22
Scraping Page: 23
Scraping Page: 24
Scraping Page: 25
Scraping Page: 26
Scraping Page: 27
Scraping Page: 28
Scraping Page: 29
Scraping Page: 30


In [36]:
len(sem_urls)
len(sem_job_title)
len(sem_info)

749

749

749

In [0]:
sem = []
base_link = 'http://www.glassdoor.com'
for link in sem_urls:
    sem.append(base_link + link)

In [38]:
sem_df = pd.DataFrame({'Job_Title':sem_job_title,"INFO":sem_info,"URLs":sem})
sem_df = sem_df[~(sem_df['Job_Title'].str.contains("None"))]
sem_df = sem_df[~(sem_df['INFO'].str.contains("None"))]
sem_df['Company_Name'] = sem_df['INFO'].apply(lambda x: x.strip().split("–")[0].strip())
sem_df['Location'] = sem_df['INFO'].apply(lambda x: x.strip().split("–")[1].strip())
sem_df = sem_df.drop("INFO",axis=1)
sem_df = sem_df.reset_index(drop=True)
sem_df.head()

,Job_Title,URLs,Company_Name,Location
0,Digital Marketing Recruiter,http://www.glassdoor.com/partner/jobListing.ht...,Jellyfish,"Baltimore, MD"
1,Marketing Associate,http://www.glassdoor.com/partner/jobListing.ht...,Entefy,"Palo Alto, CA"
2,Digital Marketing Manager,http://www.glassdoor.com/partner/jobListing.ht...,Clayco,"Saint Louis, MO"
3,Manager of Digital Data & Analytics,http://www.glassdoor.com/partner/jobListing.ht...,Chattem,"Chattanooga, TN"
4,Digital Marketing Specialist,http://www.glassdoor.com/partner/jobListing.ht...,Trupanion,"Seattle, WA"


In [39]:
sem_df = sem_df.drop_duplicates(subset=['Company_Name']).reset_index(drop=True)
len(sem_df)
sem_df.head()
sem_df['Job_Title'] = sem_df['Job_Title'].apply(lambda x: x if "Marketing" in x else 'None')
sem_df = sem_df[sem_df['Job_Title'] != 'None'].reset_index(drop=True)
len(sem_df)
sem_df.head()

563

,Job_Title,URLs,Company_Name,Location
0,Digital Marketing Recruiter,http://www.glassdoor.com/partner/jobListing.ht...,Jellyfish,"Baltimore, MD"
1,Marketing Associate,http://www.glassdoor.com/partner/jobListing.ht...,Entefy,"Palo Alto, CA"
2,Digital Marketing Manager,http://www.glassdoor.com/partner/jobListing.ht...,Clayco,"Saint Louis, MO"
3,Manager of Digital Data & Analytics,http://www.glassdoor.com/partner/jobListing.ht...,Chattem,"Chattanooga, TN"
4,Digital Marketing Specialist,http://www.glassdoor.com/partner/jobListing.ht...,Trupanion,"Seattle, WA"


354

,Job_Title,URLs,Company_Name,Location
0,Digital Marketing Recruiter,http://www.glassdoor.com/partner/jobListing.ht...,Jellyfish,"Baltimore, MD"
1,Marketing Associate,http://www.glassdoor.com/partner/jobListing.ht...,Entefy,"Palo Alto, CA"
2,Digital Marketing Manager,http://www.glassdoor.com/partner/jobListing.ht...,Clayco,"Saint Louis, MO"
3,Digital Marketing Specialist,http://www.glassdoor.com/partner/jobListing.ht...,Trupanion,"Seattle, WA"
4,"Digital Marketing, Senior Associate",http://www.glassdoor.com/partner/jobListing.ht...,Remitly,"Seattle, WA"


In [40]:
sem_df['Contact Email'] = np.nan
sem_df['Contact Name'] = np.nan
sem_df['Contact Position'] = np.nan
sem_df['Linkedin Sent Date'] = np.nan
sem_df['Date 1st Email Sent'] = np.nan
sem_df['date'] = str(datetime.datetime.now())
sem_df.head()
len(sem_df)

,Job_Title,URLs,Company_Name,Location,Contact Email,Contact Name,Contact Position,Linkedin Sent Date,Date 1st Email Sent,date
0,Digital Marketing Recruiter,http://www.glassdoor.com/partner/jobListing.ht...,Jellyfish,"Baltimore, MD",NaN,NaN,NaN,NaN,NaN,2018-08-15 17:46:12.486863
1,Marketing Associate,http://www.glassdoor.com/partner/jobListing.ht...,Entefy,"Palo Alto, CA",NaN,NaN,NaN,NaN,NaN,2018-08-15 17:46:12.486863
2,Digital Marketing Manager,http://www.glassdoor.com/partner/jobListing.ht...,Clayco,"Saint Louis, MO",NaN,NaN,NaN,NaN,NaN,2018-08-15 17:46:12.486863
3,Digital Marketing Specialist,http://www.glassdoor.com/partner/jobListing.ht...,Trupanion,"Seattle, WA",NaN,NaN,NaN,NaN,NaN,2018-08-15 17:46:12.486863
4,"Digital Marketing, Senior Associate",http://www.glassdoor.com/partner/jobListing.ht...,Remitly,"Seattle, WA",NaN,NaN,NaN,NaN,NaN,2018-08-15 17:46:12.486863


354

## SEO

In [41]:
seo_urls = []
seo_info = []
seo_job_title = []
user_agent = UserAgent()

for i in range(30):
    print("Scraping Page:",i+1)
    main_url = 'https://www.glassdoor.com/Job/us-seo-jobs-SRCH_IL.0,2_IN1_KO3,6_IP{}.htm?fromAge=1'.format(i+1)

    page = requests.get(main_url,headers={'user-agent':user_agent.chrome})

    soup = BeautifulSoup(page.content,'lxml')

    all_divs = soup.select('li.jl div div div a.jobLink')
    if all_divs != []:
        for div in all_divs:
            seo_urls.append(div['href'])
            seo_job_title.append(div.get_text())

    sleep(np.random.randint(5))
    

    all_divs1 = soup.select('li.jl div div.flexbox.empLoc div')
    if all_divs1 != []:
        for div in all_divs1:
            if div.get_text() != 'New' and div.get_text() != "We're Hiring" and div.get_text() != "EASY APPLY" and div.get_text() != "Hot" and div.get_text() != 'Award Winner':
               seo_info.append(div.get_text())

                      
    sleep(np.random.randint(5))

Scraping Page: 1
Scraping Page: 2
Scraping Page: 3
Scraping Page: 4
Scraping Page: 5
Scraping Page: 6
Scraping Page: 7
Scraping Page: 8
Scraping Page: 9
Scraping Page: 10
Scraping Page: 11
Scraping Page: 12
Scraping Page: 13
Scraping Page: 14
Scraping Page: 15
Scraping Page: 16
Scraping Page: 17
Scraping Page: 18
Scraping Page: 19
Scraping Page: 20
Scraping Page: 21
Scraping Page: 22
Scraping Page: 23
Scraping Page: 24
Scraping Page: 25
Scraping Page: 26
Scraping Page: 27
Scraping Page: 28
Scraping Page: 29
Scraping Page: 30


In [42]:
len(seo_urls)
len(seo_job_title)
len(seo_info)

900

900

900

In [0]:
seo = []
base_link = 'http://www.glassdoor.com'
for link in seo_urls:
    seo.append(base_link + link)

In [44]:
seo_df = pd.DataFrame({'Job_Title':seo_job_title,"INFO":seo_info,"URLs":seo})
seo_df = seo_df[~(seo_df['Job_Title'].str.contains("None"))]
seo_df = seo_df[~(seo_df['INFO'].str.contains("None"))]
seo_df['Company_Name'] = seo_df['INFO'].apply(lambda x: x.strip().split("–")[0].strip())
seo_df['Location'] = seo_df['INFO'].apply(lambda x: x.strip().split("–")[1].strip())
seo_df = seo_df.drop("INFO",axis=1)
seo_df = seo_df.reset_index(drop=True)
seo_df.head()

,Job_Title,URLs,Company_Name,Location
0,SEO Account Manager,http://www.glassdoor.com/partner/jobListing.ht...,Silverback Strategies LLC,"Alexandria, VA"
1,SEO Specialist,http://www.glassdoor.com/partner/jobListing.ht...,Veterans United Home Loans,"Columbia, MO"
2,Director of Marketing,http://www.glassdoor.com/partner/jobListing.ht...,LandaJob Advertising & Marketing Talent,"Kansas City, MO"
3,Digital Marketing Manager,http://www.glassdoor.com/partner/jobListing.ht...,Clayco,"Saint Louis, MO"
4,SEO Manager,http://www.glassdoor.com/partner/jobListing.ht...,"Digital Position, LLC","Charlotte, NC"


In [45]:
seo_df = seo_df.drop_duplicates(subset=['Company_Name']).reset_index(drop=True)
len(seo_df)
seo_df.head()
seo_df['Job_Title'] = seo_df['Job_Title'].apply(lambda x: x if "Marketing" in x else 'None')
seo_df = seo_df[seo_df['Job_Title'] != 'None'].reset_index(drop=True)
len(seo_df)
seo_df.head()

689

,Job_Title,URLs,Company_Name,Location
0,SEO Account Manager,http://www.glassdoor.com/partner/jobListing.ht...,Silverback Strategies LLC,"Alexandria, VA"
1,SEO Specialist,http://www.glassdoor.com/partner/jobListing.ht...,Veterans United Home Loans,"Columbia, MO"
2,Director of Marketing,http://www.glassdoor.com/partner/jobListing.ht...,LandaJob Advertising & Marketing Talent,"Kansas City, MO"
3,Digital Marketing Manager,http://www.glassdoor.com/partner/jobListing.ht...,Clayco,"Saint Louis, MO"
4,SEO Manager,http://www.glassdoor.com/partner/jobListing.ht...,"Digital Position, LLC","Charlotte, NC"


252

,Job_Title,URLs,Company_Name,Location
0,Director of Marketing,http://www.glassdoor.com/partner/jobListing.ht...,LandaJob Advertising & Marketing Talent,"Kansas City, MO"
1,Digital Marketing Manager,http://www.glassdoor.com/partner/jobListing.ht...,Clayco,"Saint Louis, MO"
2,Digital Marketing Specialist,http://www.glassdoor.com/partner/jobListing.ht...,JG Wentworth,"Chesterbrook, PA"
3,SEO Marketing Manager,http://www.glassdoor.com/partner/jobListing.ht...,Roku,"Los Gatos, CA"
4,Marketing Intern,http://www.glassdoor.com/partner/jobListing.ht...,Velosio,"Edison, NJ"


In [46]:
seo_df['Contact Email'] = np.nan
seo_df['Contact Name'] = np.nan
seo_df['Contact Position'] = np.nan
seo_df['Linkedin Sent Date'] = np.nan
seo_df['Date 1st Email Sent'] = np.nan
seo_df['date'] = str(datetime.datetime.now())
seo_df.head()
len(seo_df)

,Job_Title,URLs,Company_Name,Location,Contact Email,Contact Name,Contact Position,Linkedin Sent Date,Date 1st Email Sent,date
0,Director of Marketing,http://www.glassdoor.com/partner/jobListing.ht...,LandaJob Advertising & Marketing Talent,"Kansas City, MO",NaN,NaN,NaN,NaN,NaN,2018-08-15 17:49:43.549435
1,Digital Marketing Manager,http://www.glassdoor.com/partner/jobListing.ht...,Clayco,"Saint Louis, MO",NaN,NaN,NaN,NaN,NaN,2018-08-15 17:49:43.549435
2,Digital Marketing Specialist,http://www.glassdoor.com/partner/jobListing.ht...,JG Wentworth,"Chesterbrook, PA",NaN,NaN,NaN,NaN,NaN,2018-08-15 17:49:43.549435
3,SEO Marketing Manager,http://www.glassdoor.com/partner/jobListing.ht...,Roku,"Los Gatos, CA",NaN,NaN,NaN,NaN,NaN,2018-08-15 17:49:43.549435
4,Marketing Intern,http://www.glassdoor.com/partner/jobListing.ht...,Velosio,"Edison, NJ",NaN,NaN,NaN,NaN,NaN,2018-08-15 17:49:43.549435


252

## Digital Marketing Strategist

In [47]:
dig_mkt_strategist_urls = []
dig_mkt_strategist_info = []
dig_mkt_strategist_job_title = []
user_agent = UserAgent()

for i in range(30):
    print("Scraping Page:",i+1)
    main_url = 'https://www.glassdoor.com/Job/us-digital-marketing-strategist-jobs-SRCH_IL.0,2_IN1_KO3,31_IP{}.htm?fromAge=1'.format(i+1)

    page = requests.get(main_url,headers={'user-agent':user_agent.chrome})

    soup = BeautifulSoup(page.content,'lxml')

    all_divs = soup.select('li.jl div div div a.jobLink')
    if all_divs != []:
        for div in all_divs:
            dig_mkt_strategist_urls.append(div['href'])
            dig_mkt_strategist_job_title.append(div.get_text())

    sleep(np.random.randint(5))
    

    all_divs1 = soup.select('li.jl div div.flexbox.empLoc div')
    if all_divs1 != []:
        for div in all_divs1:
            if div.get_text() != 'New' and div.get_text() != "We're Hiring" and div.get_text() != "EASY APPLY" and div.get_text() != "Hot" and div.get_text() != 'Award Winner':
                dig_mkt_strategist_info.append(div.get_text())

                      
    sleep(np.random.randint(5))

Scraping Page: 1
Scraping Page: 2
Scraping Page: 3
Scraping Page: 4
Scraping Page: 5
Scraping Page: 6
Scraping Page: 7
Scraping Page: 8
Scraping Page: 9
Scraping Page: 10
Scraping Page: 11
Scraping Page: 12
Scraping Page: 13
Scraping Page: 14
Scraping Page: 15
Scraping Page: 16
Scraping Page: 17
Scraping Page: 18
Scraping Page: 19
Scraping Page: 20
Scraping Page: 21
Scraping Page: 22
Scraping Page: 23
Scraping Page: 24
Scraping Page: 25
Scraping Page: 26
Scraping Page: 27
Scraping Page: 28
Scraping Page: 29
Scraping Page: 30


In [48]:
len(dig_mkt_strategist_urls)
len(dig_mkt_strategist_job_title)
len(dig_mkt_strategist_info)

214

214

214

In [0]:
dig_mkt_strategist = []
base_link = 'http://www.glassdoor.com'
for link in dig_mkt_strategist_urls:
    dig_mkt_strategist.append(base_link + link)

In [50]:
dig_mkt_strategist_df = pd.DataFrame({'Job_Title':dig_mkt_strategist_job_title,"INFO":dig_mkt_strategist_info,"URLs":dig_mkt_strategist})
dig_mkt_strategist_df = dig_mkt_strategist_df[~(dig_mkt_strategist_df['Job_Title'].str.contains("None"))]
dig_mkt_strategist_df = dig_mkt_strategist_df[~(dig_mkt_strategist_df['INFO'].str.contains("None"))]
dig_mkt_strategist_df['Company_Name'] = dig_mkt_strategist_df['INFO'].apply(lambda x: x.strip().split("–")[0].strip())
dig_mkt_strategist_df['Location'] = dig_mkt_strategist_df['INFO'].apply(lambda x: x.strip().split("–")[1].strip())
dig_mkt_strategist_df = dig_mkt_strategist_df.drop("INFO",axis=1)
dig_mkt_strategist_df = dig_mkt_strategist_df.reset_index(drop=True)
dig_mkt_strategist_df.head()

,Job_Title,URLs,Company_Name,Location
0,Digital Strategist,http://www.glassdoor.com/partner/jobListing.ht...,Artisan Talent,"New York, NY"
1,Digital Media Planner,http://www.glassdoor.com/partner/jobListing.ht...,GMMB,"Washington, DC"
2,Digital Strategist,http://www.glassdoor.com/partner/jobListing.ht...,Crown Equipment,"New Bremen, OH"
3,Marketing Strategist,http://www.glassdoor.com/partner/jobListing.ht...,Capstone Search Group,"Frederick, MD"
4,Digital Marketing Strategist,http://www.glassdoor.com/partner/jobListing.ht...,TG,"Round Rock, TX"


In [51]:
dig_mkt_strategist_df = dig_mkt_strategist_df.drop_duplicates(subset=['Company_Name']).reset_index(drop=True)
len(dig_mkt_strategist_df)
dig_mkt_strategist_df.head()
dig_mkt_strategist_df['Job_Title'] = dig_mkt_strategist_df['Job_Title'].apply(lambda x: x if "Marketing" in x else 'None')
dig_mkt_strategist_df = dig_mkt_strategist_df[dig_mkt_strategist_df['Job_Title'] != 'None'].reset_index(drop=True)
len(dig_mkt_strategist_df)
dig_mkt_strategist_df.head()

186

,Job_Title,URLs,Company_Name,Location
0,Digital Strategist,http://www.glassdoor.com/partner/jobListing.ht...,Artisan Talent,"New York, NY"
1,Digital Media Planner,http://www.glassdoor.com/partner/jobListing.ht...,GMMB,"Washington, DC"
2,Digital Strategist,http://www.glassdoor.com/partner/jobListing.ht...,Crown Equipment,"New Bremen, OH"
3,Marketing Strategist,http://www.glassdoor.com/partner/jobListing.ht...,Capstone Search Group,"Frederick, MD"
4,Digital Marketing Strategist,http://www.glassdoor.com/partner/jobListing.ht...,TG,"Round Rock, TX"


91

,Job_Title,URLs,Company_Name,Location
0,Marketing Strategist,http://www.glassdoor.com/partner/jobListing.ht...,Capstone Search Group,"Frederick, MD"
1,Digital Marketing Strategist,http://www.glassdoor.com/partner/jobListing.ht...,TG,"Round Rock, TX"
2,Digital Marketing Strategist,http://www.glassdoor.com/partner/jobListing.ht...,Bay Bird,"San Diego, CA"
3,Digital Marketing Specialist,http://www.glassdoor.com/partner/jobListing.ht...,JG Wentworth,"Chesterbrook, PA"
4,Digital Marketing Specialist,http://www.glassdoor.com/partner/jobListing.ht...,Trupanion,"Seattle, WA"


In [52]:
dig_mkt_strategist_df['Contact Email'] = np.nan
dig_mkt_strategist_df['Contact Name'] = np.nan
dig_mkt_strategist_df['Contact Position'] = np.nan
dig_mkt_strategist_df['Linkedin Sent Date'] = np.nan
dig_mkt_strategist_df['Date 1st Email Sent'] = np.nan
dig_mkt_strategist_df['date'] = str(datetime.datetime.now())
dig_mkt_strategist_df.head()
len(dig_mkt_strategist_df)

,Job_Title,URLs,Company_Name,Location,Contact Email,Contact Name,Contact Position,Linkedin Sent Date,Date 1st Email Sent,date
0,Marketing Strategist,http://www.glassdoor.com/partner/jobListing.ht...,Capstone Search Group,"Frederick, MD",NaN,NaN,NaN,NaN,NaN,2018-08-15 17:53:02.535849
1,Digital Marketing Strategist,http://www.glassdoor.com/partner/jobListing.ht...,TG,"Round Rock, TX",NaN,NaN,NaN,NaN,NaN,2018-08-15 17:53:02.535849
2,Digital Marketing Strategist,http://www.glassdoor.com/partner/jobListing.ht...,Bay Bird,"San Diego, CA",NaN,NaN,NaN,NaN,NaN,2018-08-15 17:53:02.535849
3,Digital Marketing Specialist,http://www.glassdoor.com/partner/jobListing.ht...,JG Wentworth,"Chesterbrook, PA",NaN,NaN,NaN,NaN,NaN,2018-08-15 17:53:02.535849
4,Digital Marketing Specialist,http://www.glassdoor.com/partner/jobListing.ht...,Trupanion,"Seattle, WA",NaN,NaN,NaN,NaN,NaN,2018-08-15 17:53:02.535849


91

## Merge

In [53]:
data = pd.concat([vpmkt_df,digitalmkt_df,seo_df,ecommerce_df,ppc_df,digitalmkt_asso_df,sem_df,dig_mkt_strategist_df])
data.head()

,Job_Title,URLs,Company_Name,Location,Contact Email,Contact Name,Contact Position,Linkedin Sent Date,Date 1st Email Sent,date
0,Director of Marketing,http://www.glassdoor.com/partner/jobListing.ht...,LandaJob Advertising & Marketing Talent,"Kansas City, MO",NaN,NaN,NaN,NaN,NaN,2018-08-15 17:28:16.510522
1,VP Marketing,http://www.glassdoor.com/partner/jobListing.ht...,Lai Ventures - For Early Stage Client,"Scottsdale, AZ",NaN,NaN,NaN,NaN,NaN,2018-08-15 17:28:16.510522
2,VP of eCommerce Marketing,http://www.glassdoor.com/partner/jobListing.ht...,RealTruck,"Ocala, FL",NaN,NaN,NaN,NaN,NaN,2018-08-15 17:28:16.510522
3,Marketing Analyst,http://www.glassdoor.com/partner/jobListing.ht...,Owlet Baby Care,"Lehi, UT",NaN,NaN,NaN,NaN,NaN,2018-08-15 17:28:16.510522
4,"VP, Marketing",http://www.glassdoor.com/partner/jobListing.ht...,Walt Disney Company,"Burbank, CA",NaN,NaN,NaN,NaN,NaN,2018-08-15 17:28:16.510522


In [54]:
len(data)
data = data.drop_duplicates(subset=['Company_Name']).reset_index(drop=True)


2197

In [55]:
len(data)

942

In [56]:
data['Company_Name'] = data['Company_Name'].apply(lambda x: 'None' if x in Excluded_List else x)
data = data[~(data['Company_Name'].str.contains("None"))]
len(data)

892

In [57]:
len(data)
data = data[~data['Job_Title'].str.contains("|".join(Excluded_jobs),flags=re.IGNORECASE)]
data = data[~data['Company_Name'].str.contains("|".join(Big_Company_names),flags=re.IGNORECASE)]
data = data.reset_index(drop=True)

892

In [58]:
len(data)

139

In [0]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())


In [0]:
worksheet = gc.open('Business Development')
glassdoor = worksheet.worksheet("Glassdoor_Database")
glassdoor = glassdoor.get_all_values()
glassdoor_df = pd.DataFrame.from_records(glassdoor)
glassdoor_df.columns = glassdoor_df.iloc[0]
glassdoor_df = glassdoor_df.iloc[1:]

In [61]:
new_data = pd.concat([glassdoor_df,data])
new_data = new_data.reset_index(drop=True)
new_data.tail()

,Company_Name,Contact Email,Contact Name,Contact Position,Date 1st Email Sent,Job_Title,Linkedin Sent Date,Location,States,URLs,date
3165,Architecture/Interior Design Firm,NaN,NaN,NaN,NaN,Administrative / Marketing Assistant,NaN,"New York, NY",NaN,http://www.glassdoor.com/partner/jobListing.ht...,2018-08-15 17:42:31.679386
3166,CreativeLive,NaN,NaN,NaN,NaN,Paid Marketing Strategist,NaN,"San Francisco, CA",NaN,http://www.glassdoor.com/partner/jobListing.ht...,2018-08-15 17:53:02.535849
3167,Hackensack Meridian Health,NaN,NaN,NaN,NaN,"Marketing Strategist, HUMC",NaN,"Hackensack, NJ",NaN,http://www.glassdoor.com/partner/jobListing.ht...,2018-08-15 17:53:02.535849
3168,SEDAA,NaN,NaN,NaN,NaN,Marketing strategist,NaN,"San Francisco, CA",NaN,http://www.glassdoor.com/partner/jobListing.ht...,2018-08-15 17:53:02.535849
3169,Ameris Bank,NaN,NaN,NaN,NaN,Mortgage Marketing Strategist,NaN,"Marietta, GA",NaN,http://www.glassdoor.com/partner/jobListing.ht...,2018-08-15 17:53:02.535849


In [62]:
len(new_data)

3170

In [63]:
len(new_data)
new_data = new_data.drop_duplicates(subset=['Company_Name']).reset_index(drop=True)


3170

In [64]:
len(new_data)

3118

In [0]:
import gspread_dataframe as gd
a = worksheet.worksheet("Glassdoor_Database")
gd.set_with_dataframe(a,new_data)